# How to sign your results

**In this notebook we collect necessary code snippets to sign messages. They are not meant for production or anything like this. So it is really just a public notepad that we are saving here for the moment.**

Let us start with an import of the relevant functions

In [1]:
# for key generation etc
from cryptography.hazmat.primitives.asymmetric.ed25519 import (
    Ed25519PrivateKey,
    Ed25519PublicKey,
)
from cryptography.exceptions import InvalidSignature

# to save the private key
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)

# to store the public key
from cryptography.hazmat.primitives.serialization import PublicFormat

# to load the private and public key
from cryptography.hazmat.primitives.serialization import (
    load_pem_private_key,
    load_pem_public_key,
)

import base64
from sqooler.security import JWSHeader, JWSDict, payload_to_base64url

The following is the vanilla example from the [official documentation](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed25519/).

In [2]:
private_key = Ed25519PrivateKey.generate()

signature = private_key.sign(b"my authenticated message")

public_key = private_key.public_key()

# Raises InvalidSignature if verification fails

public_key.verify(signature, b"my authenticated message")

Really nice, but now we have to wonder about:

- How to generate and save the key pair ?
- How to store the signature ?
- How to load the keys at the appropriate time ?

# Storing and loading the private key

It would seem to be quite some decision to be made here. We will go the the PEM format, which seems to be fairly broadly used. I am not sure at all what the format changes, but whatever... To be seen...

In [2]:
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

# to save the private key
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    PrivateFormat,
    NoEncryption,
)

private_key = Ed25519PrivateKey.generate()
private_key_file_name = "private_key_test.pem"
private_bytes = private_key.private_bytes(
    encoding=Encoding.PEM,
    format=PrivateFormat.PKCS8,
    encryption_algorithm=NoEncryption(),
)

with open(private_key_file_name, "wb") as pem_file:
    pem_file.write(private_bytes)

Another approach to store the private key is to serialize it with base64 and then simply safe it.


In [4]:
import base64

private_key = Ed25519PrivateKey.generate()
private_bytes = private_key.private_bytes_raw()

private_b64 = base64.urlsafe_b64encode(private_bytes).decode("utf-8")

print(private_b64)

_FXeTqpgiMGn6m3GjVgVdGljRgOHPrA3_9FHQyKXfoc=


And now it is time to also load the private key.

In [4]:
with open(private_key_file_name, "rb") as pem_file:
    private_bytes_loaded = pem_file.read()

private_key_loaded = load_pem_private_key(private_bytes_loaded, password=None)

## Storing and loading the public key

Let us now also find a way to store and load the public key.

In [5]:
public_key_file_name = "public_key_test.pem"

public_bytes = public_key.public_bytes(
    Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo
)

with open(public_key_file_name, "wb") as pem_file:
    pem_file.write(public_bytes)

and load it again

In [6]:
with open(public_key_file_name, "rb") as pem_file:
    public_bytes_loaded = pem_file.read()

public_key_loaded = load_pem_public_key(public_bytes_loaded)

time to test if the loading worked nicely

In [7]:
signature = private_key_loaded.sign(b"my authenticated message")
print(signature)
public_key_loaded.verify(signature, b"my authenticated message")

b'"\x18NI\x89\xba\xeb\x1e\xdd\xeb\xc7\xc5@)-IJW\xff\x06\xdf8\xbe\x0c\xf02\xf5\xf5\x1aYS\x90\x0bi\x07\x95\x97\xb7\xa0\x10r\x13\x1f\xaf>gX7\xad\xdb\xb8\xce\x16]/]\x85%\\U\x024m\x0c'


Now  let us move on to the part, where we sign the document. Quite intestingly, we will mainly sign json files. So it might be 
that `jwcrypto` should be used fairly soon anyways. But for now, we will stick to the `cryptography` library.


In [8]:
test_dict = {"name": "daniel", "age": 25}

# let us serialize the dictionary to a json string

import json

test_dict_json = json.dumps(test_dict)
print(test_dict_json)

{"name": "daniel", "age": 25}


And it is time to sign as well as to verify the signature.

In [9]:
signature = private_key_loaded.sign(test_dict_json.encode())

public_key_loaded.verify(signature, test_dict_json.encode())

Now that this works we need to decide on how to attach the signature. Let us already have a look at the result dictionary that we are having in mind at the moment.

## JWS

The JSON Web Signature (JWS) is a compact signature format that is intended for space constrained environments such as HTTP Authorization headers and URI query parameters. It is also useful in many other situations where a JSON data structure needs to be signed or MACed. A nice introduction is given in the [official documentation](https://openid.net/specs/draft-jones-json-web-signature-04.html). For transparency purposes we will follow the standard here, but only implement the necessary parts.

Some interesting and important features are:

- The whole thing has a very well defined header, which is part of the things that should be signed.
- The payload and the header are base64url encoded and concatenated with a dot.
- Both are then signed and the signature is also base64url encoded.

Let us try to see how far we want to go there.

### Header

- `alg` : The algorithm used for the signature. We will go with `Ed25519` for now.
- `kid` : The key id. This is the name of the file where the public key is stored. We will go with the name of the file without the extension for now.
- `version`: The version of the signature format. We will go with `0.1` for now.

Other things like the adress etc will have to come later

In [10]:
header = JWSHeader(kid="test_key")

header_base64 = header.to_base64url()
print(header_base64)
payload_base64 = payload_to_base64url(test_dict)
print(payload_base64)

full_message = header_base64 + b"." + payload_base64
print(full_message)

signature = private_key.sign(full_message)
signature_base64 = base64.urlsafe_b64encode(signature)
print(signature_base64)

# the full string

sjs_string = full_message + b"." + signature_base64
print(sjs_string)

constructed_jws = JWSDict(header=header, payload=test_dict, signature=signature)

b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ0eXAiOiJKV1MiLCJ2ZXJzaW9uIjoiMC4xIn0='
b'eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0='
b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ0eXAiOiJKV1MiLCJ2ZXJzaW9uIjoiMC4xIn0=.eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0='
b'qOkIFZsH3sE5WIpSRQYluAhGr_WdAtSn4dS3SpjUfqpNN8BeBON_DtzqFLIb-nACkXXts5wrZLGUn6Lx4KEIDw=='
b'eyJhbGciOiJFZDI1NTE5Iiwia2lkIjoidGVzdF9rZXkiLCJ0eXAiOiJKV1MiLCJ2ZXJzaW9uIjoiMC4xIn0=.eyJuYW1lIjogImRhbmllbCIsICJhZ2UiOiAyNX0=.qOkIFZsH3sE5WIpSRQYluAhGr_WdAtSn4dS3SpjUfqpNN8BeBON_DtzqFLIb-nACkXXts5wrZLGUn6Lx4KEIDw=='


This is starting to shape up nicely. Let us now try to sign and verify a JWS.

In [11]:
public_key_loaded.verify(signature, full_message)

So let us now assume that we have created the payload and stored the private_key in pem file. Then we can now start to put things together.

In [18]:
def sign_payload(payload: dict, private_key: Ed25519PrivateKey, kid: str) -> JWSDict:
    """
    Convert a payload to a JWS object. We will assumar that

    Args:
        payload : The payload to convert
        private_key: The private key to use for signing
        kid: The key id of the private that you use for signing.

    Returns:
        JWS : The JWS object
    """
    header = JWSHeader(kid=kid)
    header_base64 = header.to_base64url()
    payload_base64 = payload_to_base64url(payload)
    full_message = header_base64 + b"." + payload_base64

    signature = private_key.sign(full_message)
    signature_base64 = base64.urlsafe_b64encode(signature)
    return JWSDict(header=header, payload=payload, signature=signature_base64)

In [19]:
signed_pl = sign_payload(test_dict, private_key, "test_key")
print(signed_pl.model_dump_json())

{"header":{"alg":"Ed25519","kid":"test_key","typ":"JWS","version":"0.1"},"payload":{"name":"daniel","age":25},"signature":"qOkIFZsH3sE5WIpSRQYluAhGr_WdAtSn4dS3SpjUfqpNN8BeBON_DtzqFLIb-nACkXXts5wrZLGUn6Lx4KEIDw=="}


and now we also have the full method to verify the signature.

In [16]:
def verify_jws(jws_obj: JWSDict, public_key: Ed25519PublicKey) -> bool:
    """
    Verify the integraty of JWS object.

    Args:
        jws_obj : The JWS object to verify
        public_key: The public key to use for verification

    Returns:
        if the signature can be verified
    """

    signature = base64.urlsafe_b64decode(jws_obj.signature)

    header_base64 = jws_obj.header.to_base64url()
    payload_base64 = payload_to_base64url(jws_obj.payload)
    full_message = header_base64 + b"." + payload_base64

    try:
        public_key.verify(signature, full_message)
        return True
    except InvalidSignature:
        return False

In [17]:
verify_jws(signed_pl, "public_key_test")

True

In [3]:
from sqooler.security import create_jwk_pair, jwk_from_config_str

In [2]:
private_jwk, public_jwk = create_jwk_pair("test_key")

private_key_str = private_jwk.to_config_str()
print(private_key_str)

eyJ4IjoiZ2x2a3hLeVQxblJlTmFXV2hzYnJtNFpoZUp3alliWnJBZUlIcEFIbno3RT0iLCJrZXlfb3BzIjoic2lnbiIsImtpZCI6InRlc3Rfa2V5IiwiZCI6IkVKajVPeVQ0dlhtaDVJOUt2QkVUN1U0bHdDXzd4VDdUeTdoREZrajBRbW89Iiwia3R5IjoiT0tQIiwiYWxnIjoiRWREU0EiLCJjcnYiOiJFZDI1NTE5In0=


In [4]:
jwk_from_config_str(private_key_str)

JWK(x=b'glvkxKyT1nReNaWWhsbrm4ZheJwjYbZrAeIHpAHnz7E=', key_ops='sign', kid='test_key', d=b'EJj5OyT4vXmh5I9KvBET7U4lwC_7xT7Ty7hDFkj0Qmo=', kty='OKP', alg='EdDSA', crv='Ed25519')

What about the serialization of the datetime ?

In [1]:
from pydantic import BaseModel
from datetime import datetime

In [ ]:
class TestModel(BaseModel):
    name: str
    last_queued: datetime